In [4]:
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
     
def pre_processData(newsgroups_train):
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(newsgroups_train)):
        newsgroups_train[i] = newsgroups_train[i].lower()
        newsgroups_train[i] = tokenizer.tokenize(newsgroups_train[i])
    newsgroups_train = [[token for token in doc if not token.isdigit()] for doc in newsgroups_train]
    #newsgroups_train = [[token for token in doc if len(token) > 3] for doc in newsgroups_train]
    lemmatizer = WordNetLemmatizer()
    newsgroups_train = [[lemmatizer.lemmatize(token) for token in doc] for doc in newsgroups_train]
    return newsgroups_train


def remove_stopwords(documents,news_labels):
    temp_corpus = {}
    temp_label = {}
    stop_en = stopwords.words('english')
    i = 0
    for index, words in enumerate(documents):
        rwords=[]
        for word in words:
            if word not in stop_en:
                rwords.append(word)
        if rwords:
            temp_corpus[i] = rwords
            temp_label[i] = news_labels[index]
            i = i+1
    return temp_corpus,temp_label

def process_wordvectors(vocab,vectors,documents,news_labels):
    useable_vocab = 0
    unusable_vocab = 0
    temp_corpus = {}
    temp_labels = {}
    temp_unvecs = {}
    i=0
    for index in range(len(documents)):
        filter_word = []
        words = documents[index]
        temp_unvecs[i] = {}
        for word in words:
            try:
                vectors[word]
                vocab.add(word)
                filter_word.append(word)
                useable_vocab += 1
            except:
                unusable_vocab += 1
                if word in temp_unvecs[i].keys():
                    temp_unvecs[i][word] +=1
                else:
                     temp_unvecs[i][word] = 1
                continue
        if filter_word and len(words)>0 :
            temp_corpus[i] = filter_word
            temp_labels[i] = news_labels[index]
            #print("doc ",i," not vec word len ",temp_unvecs[i])
            i= i+1
    print("There are {0} words that could be convereted to word vectors in your corpus \n" \
          "There are {1} words that could NOT be converted to word vectors".format(useable_vocab, unusable_vocab))
    print("doc num: ",i)
    print("label num: ",i)
    return temp_corpus,temp_labels,temp_unvecs
import json
if __name__ == "__main__":
    wordvec_fileapth = "glove.6B.50d.txt"
    #remove = ('headers', 'footers', 'quotes')
    #fectch_corpus = fetch_20newsgroups(subset='train', remove=remove)
    #newsgroups_train = fectch_corpus.data
    #print("target_names len ",len(fectch_corpus.target_names))
    #print("newsgroups_train size ",len(newsgroups_train))
    #print("newsgroups_train type ",type(newsgroups_train) )
    newsgroups_train = list()
    news_labels = list()
    supwords_train = list()
    file_path="part_news.txt"
    #file_path="C:/Users/aa6602623/PycharmProjects/GaussianLDA/News.txt"
    with open(file_path) as fp:
        lines = fp.read().split("\n")
        for line in lines:
            if line:
                text = json.loads(line)["textCleaned"].strip()
                label = json.loads(line)["clusterNo"]
                newsgroups_train.append(text)
                news_labels.append(label)
    fp.close()
    corpus = pre_processData(newsgroups_train)
    corpus,news_labels = remove_stopwords(corpus,news_labels)
    vectors = gensim.models.KeyedVectors.load_word2vec_format(fname=wordvec_fileapth, binary=False)
    vocab = set([])
    corpus,news_labels,supwords_train =  process_wordvectors(vocab,vectors,corpus,news_labels)
    print(len(news_labels))
    print(len(corpus))

There are 68070 words that could be convereted to word vectors in your corpus 
There are 689 words that could NOT be converted to word vectors
doc num:  11108
label num:  11108
11108
11108


In [5]:
import timeit
from scipy.special import gamma, gammaln, loggamma
from numpy import log, pi, linalg, exp, e
import random
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
import sys
doc_words = {}
for docID in range(len(corpus)):
    doc_words[docID] = {}
    words = corpus[docID]
    for word in words:
        if word not in doc_words[docID].keys():
            doc_words[docID][word] = 0
        doc_words[docID][word] += 1

In [6]:
alpha = 0.03
beta = 0.03
docID_assign_z = {}
m_z = {}
n_z = {}
n_w = {}
Topics = []
docID = 0
initial_z = 0
V = set()
D = set()
beta_topic_sum = {}
beta_topic_v = {}
gamma = 0.03
gammaS = 0.0000001

D.add(docID)
docID_assign_z[docID] = initial_z
words = corpus[docID]
if initial_z not in m_z.keys():
    m_z[initial_z] = set()
m_z[initial_z].add(docID)
for word in words:
    if initial_z not in n_w.keys():
        n_w[initial_z] = 0
    if initial_z not in n_z.keys():
        n_z[initial_z] = {}
    if word not in n_z[initial_z].keys():
        n_z[initial_z][word] = 0
    n_z[initial_z][word] += 1
    n_w[initial_z] += 1
    V.add(word)

In [14]:
def sum_topic_word():
    global_important_word = []
    for k in topic_keyword.keys():
        global_important_word.extend(list(topic_keyword[k]))
    x = None
    if len(global_important_word) !=0:
        vec_dim = 0
        for word in global_important_word:
            vec_dim += 1
            if x is not None:
                x = np.row_stack((x, vectors[word]))
            else:
                x = vectors[word] 
        global_v_bar_k[0] = None
        if  vec_dim > 1:
            global_v_bar_k[0]= np.mean(x,axis = 0)[:,None]
        else:
            x = x[:,None]
            global_v_bar_k[0] = x
        x = x.T
        global_kappa_k[0] =  kappa0 + vec_dim
        global_N[0] = vec_dim
        global_C_k[0] = (x - global_v_bar_k[0]).dot((x - global_v_bar_k[0]).T)
        global_mu_k[0] = (kappa0 * mu0 + global_N[0] * global_v_bar_k[0]) / global_kappa_k[0]
        global_psi_k[0] = psi + global_C_k[0] + (kappa0*global_N[0] / global_kappa_k[0])* ( (global_v_bar_k[0] - mu0).T.dot(global_v_bar_k[0] - mu0) )
        global_nu_k[0] = nu0 + vec_dim
        shakage_v = vec_dim + 1
        global_cov_k[0] = global_psi_k[0] / (shakage_v) 
        global_inv_cov_k[0] = np.linalg.inv(global_cov_k[0])
        #global_cov_det_k[0] = np.linalg.det(global_cov_k[0])
    else:
        shakage_v = 20
        global_cov_k[0] = psi/ (shakage_v) 
        global_mu_k[0] = mu0
        global_inv_cov_k[0] = np.linalg.inv(global_cov_k[0])
        #global_cov_det_k[0] = np.linalg.det(global_cov_k[0])
    return global_important_word

In [8]:
def sampleBetaAssignment(k, word,iter,total_iter, max_word_prob):
    if beta_topic_v[k][word] == 1:
        beta_topic_sum[k] -= 1
    pBetaAllOthers = beta_topic_sum[k]
    
    log_true = (n_z[k][word] / n_w[k]) / max_word_prob
    log_false = 1 - log_true
    
    log_p = []
    if log_false <0 or log_true < 0:
        a = 1/0
    log_p.append(log_false)
    log_p.append(log_true)
    
    sum_pro=sum(log_p)
    normalized_posterior = [i/sum_pro for i in log_p]    
    update_k = np.random.choice( 2 , 1, p=normalized_posterior)[0]
    if iter ==  total_iter - 1:
        update_k = 0
        if log_false < log_true:
            update_k = 1
    if update_k == 1:
        beta_topic_v[k][word] = 1
        beta_topic_sum[k] += 1
        topic_keyword[k].add(word)
    else:
        beta_topic_v[k][word] = 0
        topic_keyword[k].discard(word)

In [9]:
compara_batch = [len(corpus)]

In [15]:
nmi_compara_batch = []
mean_each_batch_nmi = {}
iteration_nmi = []
iteration_coherence = {}
iteration_topics = {}
iteration_NMI = {}
for i_batch in compara_batch:
    start = 0
    end = 0
    total_batch = None
    if len(corpus) % i_batch == 0:
        total_batch = int(len(corpus) / i_batch)
    else:
        total_batch = int(len(corpus) / i_batch) +1
    mean_each_batch_nmi[i_batch] = []
    for batch in range(total_batch):
        ave_nmi = []
        for mean_i in range(1):
        
            docID_assign_z = {}
            m_z = {}
            n_z = {}
            n_w = {}
            Topics = []
            V = set()
            D = set()
            beta_topic_v = {}
            beta_topic_sum = {}
            topic_keyword = {}
            
            global_v_bar_k = {}
            global_C_k= {}
            global_mu_k= {}
            global_psi_k= {}
            global_nu_k= {}
            global_kappa_k= {}
            global_cov_k= {}
            global_N= {}
            global_inv_cov_k= {}
            global_cov_det_k= {}
            kappa0 = 0.01
            dim = 50
            vec_x = 1.0
            nu0 = dim
            psi = np.eye(dim)
            mu0 = np.array([vec_x for i in range(dim)])[:,None]
            
            alpha = 0.03
            #alpha = 0.003
            
            gamma = 30
            gammaS = 0.03
            #gamma = 0.03
            #gammaS = 0.0000001
            global_word = {}
            global_important_word = None
            
            end = i_batch * (batch + 1)
            if end > len(corpus):
                end = len(corpus)
            total_iter = 10
            for iter in range(total_iter):
                iteration_coherence[iter] = {}
                iteration_topics[iter] = {}
                iteration_NMI[iter] = {}
                print("iter ",iter, " total K ",len(Topics))
#                 for k in Topics:
#                     k_words = set()
#                     for word in beta_topic_v[k]:
#                         if beta_topic_v[k][word] == 1:
#                             k_words.add(word)
                    #print("k ",k , "beta_topic_sum ",beta_topic_sum[k])
                    #print("k ",k , "n_w ",n_w[k])
                #print()
                for docID in range(start,end):
                    if docID%100 == 0:
                        print("iter ",iter, " doc ",docID)
                        print("total K ",len(Topics))
#                     if docID%5000 == 0:
#                         #print("iter ",iter, " m_z ",m_z)
#                         print("iter ",iter, " total K ",Topics)
#                         print()
                    words = corpus[docID]
                    D.discard(docID)
                    if docID in docID_assign_z.keys():
                        before_k = docID_assign_z[docID]
                        m_z[before_k].discard(docID)
                        for word in words:
                            global_word[word] -= 1
                            n_z[before_k][word] -= 1
                            n_w[before_k] -=1
#                         for word in n_z[before_k].keys():
#                             if n_z[before_k][word] > 0:
#                                 sampleBetaAssignment(before_k,word)
                        k = before_k
                        max_word_prob = n_w[k]
                        if max_word_prob != 0:
                            max_word_prob = max(n_z[k].values()) / max_word_prob

                        for word in n_z[k].keys():
                                if word not in beta_topic_v[k].keys():
                                        beta_topic_v[k][word] = 0
                                if n_z[k][word] > 0: 
                                    sampleBetaAssignment(k,word,iter,total_iter,max_word_prob)
                                else:
                                    if beta_topic_v[k][word] == 1:
                                        beta_topic_sum[k] -= 1
                                        beta_topic_v[k][word] = 0  
                                        topic_keyword[k].discard(word)
                        if docID % 500 == 0:
                            global_important_word = sum_topic_word()
                    else:
                        before_k = -1
                        
                    if len(D) == 0 and len(V) == 0:
                        choose_k = 0
                        D.add(docID)
                        docID_assign_z[docID] = choose_k
                        if choose_k not in beta_topic_v.keys(): 
                            beta_topic_v[choose_k] = {}
                        if choose_k not in beta_topic_sum.keys():
                            beta_topic_sum[choose_k] = 0
                        if choose_k not in m_z.keys():
                            m_z[choose_k] = set()
                        if choose_k not in topic_keyword.keys():
                            topic_keyword[choose_k] = set()
                        m_z[choose_k].add(docID)
                        for word in words:
                            if choose_k not in n_w.keys():
                                n_w[choose_k] = 0
                            if choose_k not in n_z.keys():
                                n_z[choose_k] = {}
                            if word not in n_z[choose_k].keys():
                                n_z[choose_k][word] = 0
                            if word not in beta_topic_v[choose_k].keys():
                                beta_topic_v[choose_k][word] = 0
                            if word not in global_word.keys():
                                global_word[word] = 0
                            global_word[word] += 1
                            n_z[choose_k][word] += 1
                            n_w[choose_k] += 1
                            V.add(word)
                        if choose_k not in Topics:
                            Topics.append(choose_k)
#                         if choose_k == Topics:
#                             Topics += 1  
                    else:
                        log_pro = []
                        
                        must_update_flag = 0
                        update_pro = 1
                        not_update_pro = 1
                        if_update_k = []
                        for word in words:
                            if word in global_important_word:
                                must_update_flag = 1
                                break
                            x = vectors[word][:,None]
                            LLcomp = (x - global_mu_k[0]).T.dot(global_inv_cov_k[0]).dot((x - global_mu_k[0])) 
                            pro = stats.chisqprob(LLcomp, dim)[0][0]
                            update_pro *= pro
                            not_update_pro *= (1 - pro)
                            
                        if must_update_flag == 0:
                            if_update_k.append(not_update_pro)
                            if_update_k.append(update_pro)
                            sum_pro=sum(if_update_k)
                            normalized_posterior = [i/sum_pro for i in if_update_k]    
                            update_k = np.random.choice( 2 , 1, p=normalized_posterior)[0]
                        else:
                            update_k = 1
                        
                        choose_k = None
                        if update_k == 1:
                            for k in Topics:
                                pro_k = len(m_z[k])
                                if pro_k == 0:
                                    log_pro.append(0)
                                else:
                                    i = 0
                                    for word in words:
                                        if word not in n_z[k].keys():
                                            n_z[k][word] = 0
                                        bias_flag = 0
                                        if word in beta_topic_v[k].keys():
                                            bias_flag = beta_topic_v[k][word]
                                        for j in range(doc_words[docID][word]):
                                            #pro_k *= (n_z[k][word]+ beta + j) / (n_w[k] + len(V)*beta + i) 
                                            pro_k *= (n_z[k][word]+ bias_flag*gamma + gammaS +j)/(n_w[k]+beta_topic_sum[k]*gamma+len(V)*gammaS +i) 
                                            i += 1
                                    log_pro.append(pro_k)
                            sum_pro=sum(log_pro)
                            normalized_posterior = [i/sum_pro for i in log_pro]  
                            select_k = None
                            if iter == (total_iter - 1):
                                select_k = normalized_posterior.index(max(normalized_posterior))

                            else:
                                select_k = np.random.choice( len(Topics) , 1, p=normalized_posterior)[0]   
                            choose_k = Topics[select_k]
                        else:
                            choose_k = np.max(Topics) + 1
                       
                            
                        D.add(docID)
                        docID_assign_z[docID] = choose_k
                        if choose_k not in m_z.keys():
                            m_z[choose_k] = set()
                        m_z[choose_k].add(docID)
                        if choose_k not in beta_topic_v.keys(): 
                            beta_topic_v[choose_k] = {}
                        if choose_k not in beta_topic_sum.keys():
                            beta_topic_sum[choose_k] = 0
                        if choose_k not in topic_keyword.keys():
                            topic_keyword[choose_k] = set()
                        for word in words:
                            if choose_k not in n_w.keys():
                                n_w[choose_k] = 0
                            if choose_k not in n_z.keys():
                                n_z[choose_k] = {}
                            if word not in n_z[choose_k].keys():
                                n_z[choose_k][word] = 0
                            if word not in beta_topic_v[choose_k].keys():
                                beta_topic_v[choose_k][word] = 0
                            if word not in global_word.keys():
                                global_word[word] = 0
                            global_word[word] += 1
                            n_z[choose_k][word] += 1
                            n_w[choose_k] += 1
                            V.add(word)
                        if choose_k not in Topics:
                            Topics.append(choose_k)
                            
                    count_k = []
                    for k in Topics:
                        if k in m_z.keys() and len(m_z[k]) == 0:
                            m_z.pop(k, None)
                            n_z.pop(k, None)
                            n_w.pop(k, None)
                            beta_topic_v.pop(k, None)
                            beta_topic_sum.pop(k, None)
                            beta_topic_v.pop(k, None)
                            topic_keyword.pop(k, None)
                            count_k.append(k)
                    for k in count_k:
                        Topics.remove(k)   
                        
                    k = choose_k
                    max_word_prob = n_w[k]
                    if max_word_prob != 0:
                        max_word_prob = max(n_z[k].values()) / max_word_prob

                    for word in n_z[k].keys():
                            if word not in beta_topic_v[k].keys():
                                    beta_topic_v[k][word] = 0
                            if n_z[k][word] > 0: 
                                sampleBetaAssignment(k,word,iter,total_iter,max_word_prob)
                            else:
                                if beta_topic_v[k][word] == 1:
                                    beta_topic_sum[k] -= 1
                                    beta_topic_v[k][word] = 0   
                                    topic_keyword[k].discard(word)
                    if docID % 100 == 0:
                        global_important_word = sum_topic_word()                
#                 for k in Topics:
#                     max_word_prob = n_w[k]
#                     if max_word_prob != 0:
#                         max_word_prob = max(n_z[k].values()) / max_word_prob

#                     for word in n_z[k].keys():
#                             if word not in beta_topic_v[k].keys():
#                                     beta_topic_v[k][word] = 0
#                             if n_z[k][word] > 0: 
#                                 sampleBetaAssignment(k,word,iter,total_iter,max_word_prob)
#                             else:
#                                 if beta_topic_v[k][word] == 1:
#                                     beta_topic_sum[k] -= 1
#                                     beta_topic_v[k][word] = 0
                #spiltDocInTopic()
#                 for k in Topics:
#                     flag = 0
#                     print("k ", k)
#                     for word in beta_topic_v[k].keys():
#                         if beta_topic_v[k][word] == 1:
#                             flag = 1
#                             print("import word:",word)
#                     print()
                
                from sklearn.metrics.cluster import normalized_mutual_info_score
                nmi_sample = []
                nmi_result = []
                for key, value in news_labels.items():
                    if key < end and key >= start:
                        nmi_sample.append(value)
                        nmi_result.append(docID_assign_z[key])
                print("iter ",iter," ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
                iteration_nmi.append(normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
                
                threshold_loop = [0, 3, 5, 10, 15, 20]
                for threshold in threshold_loop:
                    count = 0
                    k_count = set()
                    for k in m_z.keys():
                        if len(m_z[k]) >= threshold:
                            count += 1
                            k_count.add(k)
                    import operator
                    post_topic_word = {}
                    for k in k_count:
                        post_topic_word[k] = {}
                        for word in V:
                            pro = None
                            if word in beta_topic_v[k] and beta_topic_v[k][word] == 1:
                                pro = (n_z[k][word]+ beta_topic_v[k][word]*gamma + gammaS) / (n_w[k] + beta_topic_sum[k]*gamma + len(V)*beta) 
                            elif word in n_z[k]:
                                pro = (n_z[k][word]+ gammaS) / (n_w[k] + beta_topic_sum[k]*gamma + len(V)*beta) 
                            else:
                                pro = (gammaS) / (n_w[k] + beta_topic_sum[k]*gamma + len(V)*beta) 
                            post_topic_word[k][word] = pro
                        post_topic_word[k] = sorted(post_topic_word[k].items(), key=operator.itemgetter(1), reverse=True)
                    top = 10
                    topic_coherence = []
                    for k in k_count:
                        top_words = []
                        pro = 0
                        for i in range(top):
                            top_words.append( post_topic_word[k][i][0] )

                        for i in range(len(top_words)-1):
                            for j in range(i+1,len(top_words) ):
                                count_i = 0
                                count_ij = 0
                                for docID in range( len(corpus) ):
                                    if top_words[i] in corpus[docID]:
                                        count_i += 1
                                        if top_words[j] in corpus[docID]:
                                            count_ij += 1
                                pro += log( (count_ij + 1) / count_i )
                        topic_coherence.append(pro)
                    iteration_coherence[iter][threshold] = np.mean(topic_coherence)
                    iteration_topics[iter][threshold] = count
                    
                    nmi_sample = []
                    nmi_result = []
                    for k in k_count:
                        for docID in m_z[k]:
                            nmi_sample.append(news_labels[docID])
                            nmi_result.append(docID_assign_z[docID])
                    iteration_NMI[iter][threshold] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
                    
                print("iter coherence: ",iteration_coherence)
                print("iter topics: ",iteration_topics)
                print("iter NMI: ",iteration_NMI)
                
                
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            ave_nmi.append(normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
        mean_each_batch_nmi[i_batch].append(np.mean(ave_nmi))
        start = end 
        if batch%10 == 0:
            print("i_batch",i_batch , "batch ",batch)
            print(" ave_nmi ",np.mean(ave_nmi))
            print(" mean_each_batch_nmi ",np.mean(mean_each_batch_nmi[i_batch]) ,"max ", np.max(ave_nmi) ,"min ", np.min(ave_nmi) )
            print("total K ",len(Topics))
            print("global_important_word ",len(global_important_word))
            print("iteration_nmi ",iteration_nmi)
            print("iteration_coherence ",iteration_coherence)
            print()
    nmi_compara_batch.append(np.mean(mean_each_batch_nmi[i_batch]))
    print(compara_batch)
    print(nmi_compara_batch)

iter  0  total K  0
iter  0  doc  0
total K  0
iter  0  doc  100
total K  83
iter  0  doc  200
total K  85
iter  0  doc  300
total K  85
iter  0  doc  400
total K  87
iter  0  doc  500
total K  87
iter  0  doc  600
total K  89
iter  0  doc  700
total K  91
iter  0  doc  800
total K  112
iter  0  doc  900
total K  113
iter  0  doc  1000
total K  116
iter  0  doc  1100
total K  117
iter  0  doc  1200
total K  118
iter  0  doc  1300
total K  118
iter  0  doc  1400
total K  121
iter  0  doc  1500
total K  126
iter  0  doc  1600
total K  128
iter  0  doc  1700
total K  131
iter  0  doc  1800
total K  133
iter  0  doc  1900
total K  136
iter  0  doc  2000
total K  136
iter  0  doc  2100
total K  139
iter  0  doc  2200
total K  145
iter  0  doc  2300
total K  147
iter  0  doc  2400
total K  148
iter  0  doc  2500
total K  149
iter  0  doc  2600
total K  151
iter  0  doc  2700
total K  152
iter  0  doc  2800
total K  153
iter  0  doc  2900
total K  172
iter  0  doc  3000
total K  173
iter  0  

iter  2  doc  100
total K  287
iter  2  doc  200
total K  287
iter  2  doc  300
total K  286
iter  2  doc  400
total K  287
iter  2  doc  500
total K  287
iter  2  doc  600
total K  287
iter  2  doc  700
total K  288
iter  2  doc  800
total K  288
iter  2  doc  900
total K  287
iter  2  doc  1000
total K  287
iter  2  doc  1100
total K  286
iter  2  doc  1200
total K  286
iter  2  doc  1300
total K  285
iter  2  doc  1400
total K  284
iter  2  doc  1500
total K  284
iter  2  doc  1600
total K  284
iter  2  doc  1700
total K  283
iter  2  doc  1800
total K  283
iter  2  doc  1900
total K  284
iter  2  doc  2000
total K  283
iter  2  doc  2100
total K  283
iter  2  doc  2200
total K  282
iter  2  doc  2300
total K  283
iter  2  doc  2400
total K  283
iter  2  doc  2500
total K  283
iter  2  doc  2600
total K  283
iter  2  doc  2700
total K  283
iter  2  doc  2800
total K  283
iter  2  doc  2900
total K  283
iter  2  doc  3000
total K  283
iter  2  doc  3100
total K  284
iter  2  doc  320

iter  3  doc  10900
total K  256
iter  3  doc  11000
total K  256
iter  3  doc  11100
total K  256
iter  3   0.8508576705742432
iter coherence:  {0: {0: -98.96093219347183, 3: -108.86120891020987, 5: -111.71508804127546, 10: -112.39845059686135, 15: -114.50582643968436, 20: -116.74670489467519}, 1: {0: -95.83519873293832, 3: -104.83184521867932, 5: -107.36746349945534, 10: -109.02380375809703, 15: -110.21370907158162, 20: -112.98472786518006}, 2: {0: -94.69203357000072, 3: -102.66087322298402, 5: -107.84221318748615, 10: -108.39073605602248, 15: -110.87922061101446, 20: -112.11276728682425}, 3: {0: -95.23107129425495, 3: -103.36774486270188, 5: -106.33098237337383, 10: -107.29419257714001, 15: -109.46927189200242, 20: -110.13953347769139}}
iter topics:  {0: {0: 336, 3: 257, 5: 223, 10: 171, 15: 145, 20: 125}, 1: {0: 288, 3: 225, 5: 188, 10: 158, 15: 140, 20: 124}, 2: {0: 282, 3: 216, 5: 179, 10: 152, 15: 136, 20: 122}, 3: {0: 256, 3: 202, 5: 178, 10: 153, 15: 133, 20: 120}}
iter NMI:  

iter  5  doc  3700
total K  255
iter  5  doc  3800
total K  255
iter  5  doc  3900
total K  255
iter  5  doc  4000
total K  254
iter  5  doc  4100
total K  254
iter  5  doc  4200
total K  253
iter  5  doc  4300
total K  252
iter  5  doc  4400
total K  252
iter  5  doc  4500
total K  252
iter  5  doc  4600
total K  252
iter  5  doc  4700
total K  253
iter  5  doc  4800
total K  253
iter  5  doc  4900
total K  252
iter  5  doc  5000
total K  252
iter  5  doc  5100
total K  252
iter  5  doc  5200
total K  252
iter  5  doc  5300
total K  252
iter  5  doc  5400
total K  250
iter  5  doc  5500
total K  250
iter  5  doc  5600
total K  250
iter  5  doc  5700
total K  249
iter  5  doc  5800
total K  249
iter  5  doc  5900
total K  250
iter  5  doc  6000
total K  251
iter  5  doc  6100
total K  251
iter  5  doc  6200
total K  252
iter  5  doc  6300
total K  253
iter  5  doc  6400
total K  253
iter  5  doc  6500
total K  253
iter  5  doc  6600
total K  255
iter  5  doc  6700
total K  255
iter  5 

iter  6   0.8584822233808925
iter coherence:  {0: {0: -98.96093219347183, 3: -108.86120891020987, 5: -111.71508804127546, 10: -112.39845059686135, 15: -114.50582643968436, 20: -116.74670489467519}, 1: {0: -95.83519873293832, 3: -104.83184521867932, 5: -107.36746349945534, 10: -109.02380375809703, 15: -110.21370907158162, 20: -112.98472786518006}, 2: {0: -94.69203357000072, 3: -102.66087322298402, 5: -107.84221318748615, 10: -108.39073605602248, 15: -110.87922061101446, 20: -112.11276728682425}, 3: {0: -95.23107129425495, 3: -103.36774486270188, 5: -106.33098237337383, 10: -107.29419257714001, 15: -109.46927189200242, 20: -110.13953347769139}, 4: {0: -92.98531687548478, 3: -102.25863688678186, 5: -105.21070225971121, 10: -105.6086118413773, 15: -107.27963196469581, 20: -108.93705134619867}, 5: {0: -92.63016784989662, 3: -102.83342470018262, 5: -105.39246957943122, 10: -106.97484226775802, 15: -108.72456432427951, 20: -110.59074353079154}, 6: {0: -93.43154577314756, 3: -100.7902842718818

iter  8  doc  100
total K  266
iter  8  doc  200
total K  266
iter  8  doc  300
total K  267
iter  8  doc  400
total K  267
iter  8  doc  500
total K  267
iter  8  doc  600
total K  265
iter  8  doc  700
total K  265
iter  8  doc  800
total K  265
iter  8  doc  900
total K  265
iter  8  doc  1000
total K  265
iter  8  doc  1100
total K  265
iter  8  doc  1200
total K  265
iter  8  doc  1300
total K  264
iter  8  doc  1400
total K  262
iter  8  doc  1500
total K  262
iter  8  doc  1600
total K  262
iter  8  doc  1700
total K  262
iter  8  doc  1800
total K  262
iter  8  doc  1900
total K  261
iter  8  doc  2000
total K  261
iter  8  doc  2100
total K  262
iter  8  doc  2200
total K  262
iter  8  doc  2300
total K  261
iter  8  doc  2400
total K  260
iter  8  doc  2500
total K  261
iter  8  doc  2600
total K  261
iter  8  doc  2700
total K  262
iter  8  doc  2800
total K  261
iter  8  doc  2900
total K  261
iter  8  doc  3000
total K  261
iter  8  doc  3100
total K  261
iter  8  doc  320

iter  9  doc  4400
total K  253
iter  9  doc  4500
total K  253
iter  9  doc  4600
total K  253
iter  9  doc  4700
total K  253
iter  9  doc  4800
total K  256
iter  9  doc  4900
total K  255
iter  9  doc  5000
total K  256
iter  9  doc  5100
total K  256
iter  9  doc  5200
total K  256
iter  9  doc  5300
total K  256
iter  9  doc  5400
total K  257
iter  9  doc  5500
total K  257
iter  9  doc  5600
total K  257
iter  9  doc  5700
total K  256
iter  9  doc  5800
total K  256
iter  9  doc  5900
total K  256
iter  9  doc  6000
total K  256
iter  9  doc  6100
total K  256
iter  9  doc  6200
total K  256
iter  9  doc  6300
total K  255
iter  9  doc  6400
total K  255
iter  9  doc  6500
total K  255
iter  9  doc  6600
total K  260
iter  9  doc  6700
total K  263
iter  9  doc  6800
total K  264
iter  9  doc  6900
total K  264
iter  9  doc  7000
total K  265
iter  9  doc  7100
total K  266
iter  9  doc  7200
total K  266
iter  9  doc  7300
total K  267
iter  9  doc  7400
total K  266
iter  9 

In [17]:
topic_word = {}
top_n = 10
for topic_num in n_z.keys():
    top_n_word = sorted(n_z[topic_num].keys(), key=lambda i: n_z[topic_num][i], reverse=True)[:top_n]
    topic_word[topic_num] = [i for i in top_n_word]

In [18]:
for i in topic_word.keys():
    print("Topic",topic_word[i][0],":",topic_word[i])

Topic coalition : ['coalition', 'german', 'merkel', 'deal', 'spd', 'party', 'germany', 'consumer', 'reach', 'social']
Topic aarushi : ['aarushi', 'murder', 'talwar', 'case', 'hemraj', 'talwars', 'nupur', 'rajesh', 'daughter', 'life']
Topic pm : ['pm', 'nawaz', 'nuclear', 'karachi', 'sharif', 'project', 'today', 'plant', 'lay', 'foundation']
Topic resigns : ['resigns', 'supermarket', 'collapse', 'pm', 'latvian', 'latvia', 'roof', 'prime', 'minister', 'disaster']
Topic china : ['china', 'measure', 'bolster', 'east', 'sea', 'claim', 'warns', 'defensive', 'abbott', 'indonesian']
Topic macy : ['macy', 'stay', 'giant', 'mover', 'grounded', 'balloon', 'remove', 'autonomy', 'advantage', 'midday']
Topic karzai : ['karzai', 'security', 'afghan', 'deal', 'troop', 'afghanistan', 'pact', 'sign', 'rice', 'agreement']
Topic cameron : ['cameron', 'benefit', 'eu', 'migrant', 'david', 'britain', 'curb', 'rule', 'uk', 'pm']
Topic israel : ['israel', 'eu', 'israeli', 'palestinian', 'bank', 'west', 'settle